In [115]:
import pandas as pd
import tiktoken
from langchain_openai import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate
from dotenv import load_dotenv
from tqdm import tqdm

tqdm.pandas()
load_dotenv()

True

In [ ]:
sc_101_deepl_df = pd.read_csv('../datasets/social-chem-101/social-chem-101_care-harm_4_deepl_translated.csv')

In [120]:
# system_message = SystemMessagePromptTemplate.from_template( """
# Ти досвідчений перекладач з англійської на українську, який враховує котекст та відмінки.
# Твоя задача - переклад тексту з англійської на українську.
                                                           
# Важливі вимоги до перекладу:
# - Уважно проаналізуй оригінальний текст.
# - Збереження змісту: переклад повинен точно передавати значення оригіналу.
# - Збереження стилю: переклад має зберігати стиль і тон оригінального тексту.
# - Граматична правильність: переклад має бути природним і відповідати нормам української мови.                                                            
# """
# )
system_message = SystemMessagePromptTemplate.from_template( """
Ти досвідчений перекладач з англійської на українську.
Твоя задача - переклад тексту з англійської на українську.                                                         
"""
)

translation_refinement_humam_message =HumanMessagePromptTemplate.from_template( """
Оригінальний текст: 
{original_text} 

Поверни лише переклад. 
"""
)

chat_prompt = ChatPromptTemplate.from_messages([system_message, translation_refinement_humam_message])

In [63]:
encoding = tiktoken.get_encoding("o200k_base")

In [64]:
input = chat_prompt.format(
    original_text=sc_101_deepl_df.iloc[0]['action'],
    current_translation=sc_101_deepl_df.iloc[0]['action_ukr'],
)
print(input)

System: 
Ти досвідчений перекладач з англійської на українську, який враховує котекст та відмінки.
Твоя задача - переклад тексту з англійської на українську.
                                                           
Важливі вимоги до перекладу:
- Уважно проаналізуй оригінальний текст.
- Збереження змісту: переклад повинен точно передавати значення оригіналу.
- Збереження стилю: переклад має зберігати стиль і тон оригінального тексту.
- Граматична правильність: переклад має бути природним і відповідати нормам української мови.                                                            

Human: 
Оригінальний текст: 
Abusing children 

Поверни лише переклад. 



In [65]:
prompt_tokens = encoding.encode(input)
output_tokens = encoding.encode(sc_101_deepl_df.iloc[0]['action_ukr'])
print(f"Output tokens: {len(output_tokens)}")
print(f"Prompt tokens: {len(prompt_tokens)}")

Output tokens: 10
Prompt tokens: 175


163x3682 =~ 600k

In [66]:
llm = ChatOpenAI(model='gpt-4o')

In [67]:
def refine_translation(row):
    chain = chat_prompt | llm
    # response = chain.invoke({"original_text": row['action'], "current_translation": row['action_ukr']})
    response = chain.invoke({"original_text": row['action']})
    refined_translation = response.content.strip()
    return refined_translation

In [74]:
sc_101_deepl_df.loc[20:30, 'action_ukr_gpt'] = sc_101_deepl_df.loc[20:30,:].progress_apply(refine_translation, axis=1)

100%|██████████| 11/11 [00:08<00:00,  1.36it/s]


In [75]:
sc_101_deepl_df.loc[20:30,['action', 'action_ukr', 'action_ukr_gpt']]

,action,action_ukr,action_ukr_gpt
20,Being honest with your girlfriend about past h...,Бути чесним зі своєю дівчиною про минулі зв'яз...,Бути чесним з дівчиною щодо минулих зв'язків і...
21,Being loyal to your friends.,Бути вірним своїм друзям.,Бути вірним своїм друзям.
22,Being mad at a significant other over small th...,Злитися на другу половинку через дрібниці,Сердитися на близьку людину через дрібниці
23,Being polite.,Бути ввічливим.,Бути ввічливим.
24,Being rude to a person that is just doing thei...,"Грубіянити людині, яка просто виконує свою роб...","Бути грубим до людини, яка просто виконує свою..."
25,Being supportive of your significant other whe...,"Підтримувати свою другу половинку, коли вона в...","Підтримувати свою другу половинку, коли вона в..."
26,Being there for a family member or friend that...,Бути поруч з невиліковно хворим членом сім'ї а...,"Бути поруч із родичем або другом, який хворий ..."
27,Being too rough with girls.,Бути занадто грубим з дівчатами.,Бути занадто грубим з дівчатами.
28,Being upset if partner is not letting other pa...,"Засмучуватися, якщо партнер не дозволяє іншому...","Засмучення, якщо партнер не дозволяє іншому па..."
29,Bringing out an underage drinker.,Виводимо неповнолітнього п'яницю.,Вивести на світ неповнолітнього споживача алко...


In [ ]:
system_message2 = SystemMessagePromptTemplate.from_template( """
Ти досвідчений перекладач з англійської на українську, який враховує котекст та відмінки.
Твоя задача - покращення перекладу тексту з англійської на українську.
                                                           
Важливі вимоги до покращення:
- Аналіз: уважно проаналізуй оригінальний текст та зрозумій його зміст. Зверни увагу на фразеологізми, ідіоми та культурні особливості.
- Збереження змісту: переклад повинен передавати значення оригіналу.
- Увага до структури: переклад повинен бути зв'язним та зрозумілим.
- Правильність: переклад має бути природним і відповідати нормам української мови.
                                                          
"""
)

translation_refinement_humam_message2 =HumanMessagePromptTemplate.from_template( """
Оригінальний текст: 
{original_text} 
                                                                               
Варіанти перекладу:
{current_translations}

Поверни лише покращений переклад. 
"""
)

chat_prompt2 = ChatPromptTemplate.from_messages([system_message2, translation_refinement_humam_message2])

In [102]:
def refine_translation2(row):
    chain = chat_prompt2 | llm
    current_translations = [
        row['action_ukr'],
        row['action_ukr_gpt'],
    ]
    response = chain.invoke({"original_text": row['action'], "current_translations": current_translations})
    refined_translation = response.content.strip()
    return refined_translation

In [103]:
sc_101_deepl_df.loc[20:30, 'action_ukr_gpt2'] = sc_101_deepl_df.loc[20:30,:].progress_apply(refine_translation2, axis=1)

100%|██████████| 11/11 [00:13<00:00,  1.18s/it]


In [105]:
sc_101_deepl_df.loc[20:30,['action', 'action_ukr', 'action_ukr_gpt', 'action_ukr_gpt2']]

,action,action_ukr,action_ukr_gpt,action_ukr_gpt2
20,Being honest with your girlfriend about past h...,Бути чесним зі своєю дівчиною про минулі зв'яз...,Бути чесним з дівчиною щодо минулих зв'язків і...,Бути відвертим зі своєю дівчиною щодо колишніх...
21,Being loyal to your friends.,Бути вірним своїм друзям.,Бути вірним своїм друзям.,Залишатися вірним своїм друзям.
22,Being mad at a significant other over small th...,Злитися на другу половинку через дрібниці,Сердитися на близьку людину через дрібниці,Сердитися на кохану людину через дрібниці
23,Being polite.,Бути ввічливим.,Бути ввічливим.,Виявляти ввічливість.
24,Being rude to a person that is just doing thei...,"Грубіянити людині, яка просто виконує свою роб...","Бути грубим до людини, яка просто виконує свою...","Грубіянити людині, яка просто виконує свої обо..."
25,Being supportive of your significant other whe...,"Підтримувати свою другу половинку, коли вона в...","Підтримувати свою другу половинку, коли вона в...","Підтримувати свою кохану людину, коли вона у д..."
26,Being there for a family member or friend that...,Бути поруч з невиліковно хворим членом сім'ї а...,"Бути поруч із родичем або другом, який хворий ...","""Підтримувати невиліковно хворого члена сім'ї ..."
27,Being too rough with girls.,Бути занадто грубим з дівчатами.,Бути занадто грубим з дівчатами.,Поведінка надто груба щодо дівчат.
28,Being upset if partner is not letting other pa...,"Засмучуватися, якщо партнер не дозволяє іншому...","Засмучення, якщо партнер не дозволяє іншому па...","Засмучення, коли партнер не дозволяє іншому ма..."
29,Bringing out an underage drinker.,Виводимо неповнолітнього п'яницю.,Вивести на світ неповнолітнього споживача алко...,Виявлення неповнолітнього споживача алкоголю.


In [106]:
from langchain_anthropic import ChatAnthropic

In [125]:
import os
a_llm = ChatAnthropic(model='claude-3-5-haiku-latest')

In [126]:
def refine_translation3(row):
    chain = chat_prompt | a_llm
    # response = chain.invoke({"original_text": row['action'], "current_translation": row['action_ukr']})
    response = chain.invoke({"original_text": row['action']})
    refined_translation = response.content.strip()
    return refined_translation

In [127]:
sc_101_deepl_df.loc[20:30, 'action_ukr_claude3.5'] = sc_101_deepl_df.loc[20:30,:].progress_apply(refine_translation3, axis=1)

100%|██████████| 11/11 [00:11<00:00,  1.08s/it]


In [128]:
sc_101_deepl_df.loc[20:30,['action', 'action_ukr','action_ukr_gpt2', 'action_ukr_claude3.7', 'action_ukr_claude3.5']]

,action,action_ukr,action_ukr_gpt2,action_ukr_claude3.7,action_ukr_claude3.5
20,Being honest with your girlfriend about past h...,Бути чесним зі своєю дівчиною про минулі зв'яз...,Бути відвертим зі своєю дівчиною щодо колишніх...,Бути чесним зі своєю дівчиною щодо минулих інт...,Бути чесним з дівчиною про минулі short-time с...
21,Being loyal to your friends.,Бути вірним своїм друзям.,Залишатися вірним своїм друзям.,Бути відданим своїм друзям.,Бути вірним своїм друзям.
22,Being mad at a significant other over small th...,Злитися на другу половинку через дрібниці,Сердитися на кохану людину через дрібниці,Злитися на кохану людину через дрібниці,Бути злим на близьку людину через дрібниці
23,Being polite.,Бути ввічливим.,Виявляти ввічливість.,Бути ввічливим.,Бути ввічливим.
24,Being rude to a person that is just doing thei...,"Грубіянити людині, яка просто виконує свою роб...","Грубіянити людині, яка просто виконує свої обо...","Бути грубим до людини, яка просто виконує свою...","Бути грубим з людиною, яка просто виконує свою..."
25,Being supportive of your significant other whe...,"Підтримувати свою другу половинку, коли вона в...","Підтримувати свою кохану людину, коли вона у д...","Підтримувати свого партнера, коли він у депресії.","Підтримка вашого близького, коли він перебуває..."
26,Being there for a family member or friend that...,Бути поруч з невиліковно хворим членом сім'ї а...,"""Підтримувати невиліковно хворого члена сім'ї ...",Бути поруч із смертельно хворим членом родини ...,"Бути поруч з членом сім'ї або другом, який нев..."
27,Being too rough with girls.,Бути занадто грубим з дівчатами.,Поведінка надто груба щодо дівчат.,Бути занадто грубим з дівчатами.,Бути надто грубим з дівчатами.
28,Being upset if partner is not letting other pa...,"Засмучуватися, якщо партнер не дозволяє іншому...","Засмучення, коли партнер не дозволяє іншому ма...","Засмучуватися, якщо партнер не дозволяє іншому...","Бути засудженим, якщо партнер не дозволяє іншо..."
29,Bringing out an underage drinker.,Виводимо неповнолітнього п'яницю.,Виявлення неповнолітнього споживача алкоголю.,"Викриття неповнолітнього, який вживає алкоголь.",Виведення неповнолітнього п'яниці.


In [131]:
# subset_df = sc_101_deepl_df[['action', 'action_ukr', 'action_ukr_gpt', 'action_ukr_gpt2', 'action_ukr_claude3.7', 'action_ukr_claude3.5']].head(30)
subset_df2 = sc_101_deepl_df.head(30)
subset_df2.to_csv('test_translation2.csv', index=False)

In [132]:
import anthropic

client = anthropic.Anthropic()

response = client.messages.count_tokens(
    model="claude-3-7-sonnet-latest",
    system="""
Ти досвідчений перекладач з англійської на українську.
Твоя задача - переклад тексту з англійської на українську.    
""",
    messages=[{
        "role": "user",
        "content": """
Оригінальний текст: 
Bringing out an underage drinker. 

Поверни лише переклад. 
"""
    }],
)

print(response.json())

{"input_tokens":85}


/var/folders/h6/h54l3p8j0pv1w179xqnxjknm0000gp/T/ipykernel_23798/4173039767.py:22: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  print(response.json())
